In [ ]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage, BaseOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt # load prompt from json/yaml

In [ ]:
llm = ChatOpenAI(
    temperature=0.1, # creativity
    streaming=True, # stream response
    callbacks=[StreamingStdOutCallbackHandler()] # print streaming response
) # default: gpt-3.5-turbo

In [ ]:
# predict text
llm.predict("How many planets are there?")

# predict text using template
template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?")
"""
template = PromptTemplate(
    template="What is the distance between {country_a} and {country_b}?",
    input_variables=["country_a", "country_b"],
)
template = load_prompt("prompt.json")
"""
prompt = template.format(country_a="Mexico", country_b="Thailand")
print(prompt)
llm.predict(prompt)

In [ ]:
# predict from messages
messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."), # default context
    AIMessage(content="Ciao, mi chiamo Paolo!"), # conversation history
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, What is your name?")
]
llm.predict_messages(messages)

# predict from messages using template
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, What is your name?")
])
prompt = template.format_messages(
    language="Greel",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)
print(template)
llm.predict_messages(prompt)

In [ ]:
# output parser (change response)
class CommaOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        items = text.strip().split(",")
        return list(map(str.strip, items))
parser = CommaOutputParser()

## example
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comman seperated list of max {max_items} in lower case. Do NOT reply with anything else."),
    ("human", "{question}")
])
prompt = template.format_messages(max_items=10, question="What are the planets?")
answer = llm.predict_messages(prompt)
parser.parse(answer.content)

In [ ]:
# LCEL (input | output)
"""
component     |     input          |    output
-------------------------------------------------------
template      |     **dict**       |    prompt value
llm           |     prompt value   |    chat message
parser        |     chat message   |    **parsed message**
"""
chain = template | llm | parser # template: .format_messages -> llm: .predict_messages -> parser: .parse

chain.invoke({
    # template input
    "max_items": 5,
    "question": "What are the pokemons?"
})